<a href="https://colab.research.google.com/github/BrotherKim/AI599/blob/master/231211_final_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#model setting

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q grobid-client langchain openai faiss-cpu tiktoken sentence-transformers chromadb matplotlib

In [ ]:
!cp -n "/content/drive/My Drive/FinalData/Fine_all.zip" "./"
!unzip -n -qq "/content/Fine_all.zip" -d '/content/FAISS_Fine'
!cp -n "/content/drive/My Drive/FinalData/Delay_all.zip" "./"
!unzip -n -qq "/content/Delay_all.zip" -d '/content/FAISS_Delay'

# streamlit setting

In [ ]:
!pip install streamlit
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 1.6 MB/s eta 0:00:00
  Attempting uninstall: attrs
    Found existing installation: attrs 21.4.0
    Uninstalling attrs-21.4.0:
      Successfully uninstalled attrs-21.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
grobid-client 0.8.3 requires attrs<22.0.0,>=20.1.0, but you have attrs 23.1.0 which is incompatible.


In [ ]:
%%writefile app.py

import streamlit as st
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier

import pandas as pd
import numpy as np
import re
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

import math

# model setting
def extract_output_and_row(document):
  output_pattern = re.compile(r'output:\s*(\S+)')
  output_match = output_pattern.search(document)
  if output_match:
    extracted_output = output_match.group(1)
  else:
    extracted_output = None
  return extracted_output

def retrieve_combined_output(testset, target_rows):
  combined_output = []
  for row in target_rows:
    # 해당 row와 일치하는 testset의 'instruction' 값을 찾아 합침
    instruction = testset.loc[row, 'instruction']
    combined_output.append(instruction)
  return combined_output

def get_result_k(testset, db, query, k=3):
  # 테스트 쿼리를 검색하여 유사도 결과를 시뮬레이션하고, 각 결과에 대해 output을 찾음
  ans_list = db.similarity_search_with_score(query, k=k)
  results = []

  # 각 결과에 대해 output과 row를 results에 저장
  for result in ans_list:
    page, _ = result
    row = page.metadata.get('row')
    output = testset.loc[row, 'output']
    results.append((output, row))

  print(ans_list)
  print(results)

  # 결과가 비어있는 경우 예외 처리
  if not results:
    print("No results found.")
    return None, None

  # 결과의 빈도를 저장할 딕셔너리 생성
  output_counts = {}

  # 결과 리스트를 순회하며 빈도를 계산하여 딕셔너리에 저장
  for output, _ in results:
    if output in output_counts:
      output_counts[output] += 1
    else:
      output_counts[output] = 1

  # 딕셔너리에서 가장 빈도가 높은 output을 찾음
  final_output = max(output_counts, key=output_counts.get)

  # 최종 output으로 선택된 row들의 내용을 합침
  target_rows = [row for _, row in results if _ == final_output]
  # testset을 사용하여 'row' 값을 기준으로 'instruction'을 가져옴
  combined_output = testset.loc[target_rows, 'instruction'].tolist()

  print(final_output)
  print(combined_output)
  return final_output, combined_output

def run_with_fixed_k(query):
  hf = HuggingFaceEmbeddings(
  model_name = "jhgan/ko-sroberta-multitask",
  model_kwargs = {'device' : 'cpu'},
  encode_kwargs = {'normalize_embeddings' :True})

  testset = pd.read_csv('/content/drive/MyDrive/FinalData/1130_2004001000_85_F.csv', index_col=None)
  db_fine = FAISS.load_local("/content/FAISS_Fine/",hf)
  db_to_use = db_fine

  # 입력받은 query로 결과를 가져옴
  final_output_fine, combined_output_fine = get_result_k(testset, db_fine, query, k=1)

  # 결과가 징역이면 'delay'로 다시 실행
  if final_output_fine == '징역':
    testset = pd.read_csv('/content/drive/MyDrive/FinalData/1130_2004001000yy_85_F.csv', index_col=None)
    db_delay = FAISS.load_local("/content/FAISS_Delay/",hf)
    db_to_use = db_delay
    final_output_delay, combined_output_delay = get_result_k(testset, db_delay, query, k=1)
    return final_output_delay, combined_output_delay
  else:
    return final_output_fine, combined_output_fine


# summary setting
tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')

def bart_summarize(text):
  raw_input_ids = tokenizer.encode(text)
  input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]
  summary_ids = model.generate(torch.tensor([input_ids]), num_beams=4, length_penalty=1.0, max_length=math.trunc(len(text)*0.8), eos_token_id=1)
  return tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)

def user_input_features() :
  blood_alcohol_level = st.sidebar.slider('혈중 알코올 농도[%]', 0.00, 0.80, 0.03)
  fact = st.sidebar.text_area('사건 상세 정보', '', help='사건 당시의 사실을 입력해주세요.')
  yh = st.sidebar.text_area('과거 전력 또는 양형의 사유', '', help='과거 음주운전 사실이나 자신이 처한 상황을 설명해주세요.')
#   data = {
#     'blood_alcohol_level' : blood_alcohol_level,
#     'fact' : fact,
#     'yh' : yh,
#   }
#   f = pd.DataFrame(data, index=[0])
  return blood_alcohol_level, fact, yh

def get_summary_from_bart(b, f, y):
  raw = ''
  raw += f'혈중 알코올 농도 {b}%인 상태로 음주운전을 했다. '
  raw += f'{f}. '
  raw += f'{y}.'

  result = ''
  result = f'혈중 알코올 농도 {b}%인 상태로 음주운전을 했다. '
  result = bart_summarize(raw)
  return result

def get_type_with_summary(summary):
  t, combined = run_with_fixed_k(summary)
  return t, combined

def main():
	#st.title("Awesome Streamlit for MLDDD")
	#st.subheader("How to run streamlit from colab")
  st.write("""
  # 음주운전 형종 예측기

  음주운전 사실에 대한 입력을 넣으면 **형벌의 종류**를 예측합니다!

  """)

  st.sidebar.header('사실 입력')

  b, f, y = user_input_features()

  st.subheader("입력된 값을 확인해주세요.")
  st.write(f'혈중 알코올 농도: {b}')
  st.write(f'사건 상세 정보: {f}')
  st.write(f'과거 전력 또는 양형의 사유: {y}')

  #st.subheader("생성 모델을 통해 자연스러운 문장으로 변형합니다.")
  summary = get_summary_from_bart(b, f, y)
  #st.write(summary)

  st.subheader("변환된 문장을 통해 예측된 형종은 아래와 같습니다.")
  t, combined = get_type_with_summary(summary)
  st.write(f'예상 형종: {t}')
  for c in combined:
    st.write(f'유사 판결문: {c}')



if __name__ == '__main__':
	main()


Overwriting app.py


In [ ]:
!ngrok authtoken 2YsyBicIiaCYfYBvklsrw4l0UrZ_7f4NHXdWjNuXViBdVjKp1
# !ngrok authtoken 2ZHoiZO9Uk340y8PM94ZRnqKJsG_4mA2YKGs9C6vG5o8RUxn1
!streamlit run app.py --server.port 8052 &> /dev/null &

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok

publ_url = ngrok.connect(addr='8052')
print(publ_url)

NgrokTunnel: "https://69b6-35-221-226-203.ngrok-free.app" -> "http://localhost:8052"


# 여기에 커서 두고 이전 셀 실행!
아래에 실행 셀 2개는 streamlit과 ngrok 죽이는 명령어

In [ ]:
# 여기에 커서를 두고 이전 셀 실행

In [ ]:
!ps | grep streamlit

   2807 ?        00:00:01 streamlit


In [ ]:
!ps | grep streamlit | grep -v grep | awk '{print $1}' | xargs kill -9

In [ ]:
ngrok.kill()

test

In [ ]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')

def bart_summarize(text):
    raw_input_ids = tokenizer.encode(text)
    input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]
    summary_ids = model.generate(torch.tensor([input_ids]), num_beams=4, max_length=600, eos_token_id=1)
    return tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)

result = bart_summarize('안녕하세요!')

tokenizer_config.json:   0%|          | 0.00/295 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/682k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/109 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


pytorch_model.bin:   0%|          | 0.00/496M [00:00<?, ?B/s]

In [ ]:
print(result)

안녕하세요!
